In [12]:
# !! FINAL POLARITY CODE !!

# Google authentication key export
!export GOOGLE_APPLICATION_CREDENTIALS="/srv/Twitter2030/Data/acts-cloud-5-tljh-8e85a08eab1d-sakey.json"

from tweet_parser.tweet import Tweet
from tweet_parser.tweet_parser_errors import NotATweetError
import fileinput
import json
import string
import re
import time
import nltk
from nltk.util import ngrams
# nltk.download('universal_tagset')
from nltk.sentiment import SentimentIntensityAnalyzer

# Imports the Google Cloud client library
from google.cloud import storage

sia = SentimentIntensityAnalyzer()

# Create bucket client and authenticate via service account access key
storage_client = storage.Client.from_service_account_json('/srv/Twitter2030/Data/acts-cloud-5-tljh-8e85a08eab1d-sakey.json')

# Set bucket name, team folder name, and delimiter
BUCKET_NAME = "allteamsmlb" # Do not change
team_name = "NewTest" # Change as per team name directory, e.g., <##Name>

# Dictionaries: Key - Date    Value - Number of tweets of that sentiment
yes_cognition_dict = {}
no_cognition_dict = {}

super_list = [['NOUN', 'VERB', 'ADV'], ['NOUN', 'VERB', 'ADP'],
              ['NOUN', 'VERB', 'ADV', 'ADJ'], ['NOUN', 'VERB', 'NOUN', 'ADP'],
              ['NOUN', 'VERB', 'ADJ'], ['NOUN', 'VERB', 'ADJ', 'NOUN']]

print('El Fin')

El Fin


In [16]:
def clean_tweets(tweet):
	# remove stock market tickers like $GE
	tweet = re.sub(r'\$\w*', '', tweet)

	# remove old style retweet text "RT"
	tweet = re.sub(r'^RT[\s]+', '', tweet)

	# remove hyperlinks
	tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
	
	# remove hashtags
	# only removing the hash # sign from the word
	tweet = re.sub(r'#', '', tweet)
 
	return tweet

t0 = time.time()

object_generator = storage_client.list_blobs(BUCKET_NAME, prefix=team_name, delimiter=None)

for status_log_bucket_name in object_generator:
    #status_log_gz = status_log_bucket_name.name.split('/')
    #print(status_log_bucket_name)
    #print(status_log_gz[1])

# https://googleapis.dev/python/storage/latest/blobs.html#google.cloud.storage.blob.Blob.download_as_text
# See section on open() function about 70% down the page
    client2 = storage.Client()
    BUCKET_NAME_TWO = client2.bucket(BUCKET_NAME)
    blob = BUCKET_NAME_TWO.blob(status_log_bucket_name.name)
    #blob = BUCKET_NAME.blob(status_log_gz)

#blobcontent = blob.download_as_string()

    with blob.open('rt') as f:
        for line in f:
            try:
                tweet_dict = json.loads(line)

                tweet = Tweet(tweet_dict)

                text = clean_tweets(tweet.all_text)

                # Extracts date and verifies its a day the Sox play
                date = tweet.created_at_string[0:10]
                
                # if date in cubs_game_dates:
                tokens = nltk.word_tokenize(text) # Create a list of tokens
                tokens_tagged = nltk.pos_tag(tokens, tagset='universal')

                pos_tokens = ''

                # Loop the dict of tokens
                for token in tokens_tagged:
                    pos_tokens += token[1] + ' '

                sentence = str(pos_tokens)#'Whoever is happy will make others happy too'
                trigrams = ngrams(sentence.split(), 3)
                quadgrams = ngrams(sentence.split(), 4)

                trigram_list = []
                for item in trigrams:
                    trigram_list.append(item[0])
                    trigram_list.append(item[1])
                    trigram_list.append(item[2])

                quadgram_list = []
                for item in quadgrams:
                    quadgram_list.append(item[0])
                    quadgram_list.append(item[1])
                    quadgram_list.append(item[2])
                    quadgram_list.append(item[3])

                if trigram_list in super_list or quadgram_list in super_list:
                    if date in yes_cognition_dict.keys():
                        yes_cognition_dict[date] += 1
                    else:
                        yes_cognition_dict[date] = 1
                else:
                    if date in no_cognition_dict.keys():
                        no_cognition_dict[date] += 1
                    else:
                        no_cognition_dict[date] = 1

            except (json.JSONDecodeError, NotATweetError):
                pass
            
t1 = time.time()

print('El Fin')
print('Runtime:', str(round(t1 - t0, 2)), 'seconds')

El Fin
Runtime: 4.73 seconds


In [9]:
# Writes to text files
with open("Beta/Output/Cognition/TEST_yes.txt", 'w') as new_file: 
    for key, value in yes_cognition_dict.items(): 
        new_file.write('%s : %d\n' % (key, value))

with open("Beta/Output/Cognition/TEST_no.txt", 'w') as new_file: 
    for key, value in no_cognition_dict.items(): 
        new_file.write('%s : %d\n' % (key, value))
        
print('El Fin')

El Fin
